# Generative Adverserial Imitation Learning

Generative Adversarial Imitation Learning (GAIL) was first proposed in the paper [Generative Adversarial Imitation Learning](https://arxiv.org/abs/1606.03476) by Jonathan Ho and Stefano Ermon. The project task is to implement the Generative Adverserial Imitation Learning model for driving scenarios using the BARK-simulator.

GAIL is based on the setting of Reinforcement Learning (RL). In Reinforcment Learning, the agent interacts with the environment through its actions and receives rewards in return. The aim of the learning process is to maximize the cummulative reward by chosing the best action in all states.

As the name suggests, GAIL belongs to a smaller subgroup of RL, called Imitation Learning. In this setup the goal of the agent is to mimic an expert behavior as closely as possible. The environment awards higher rewards to expert-like behavior and smaller ones to actions which substantially deviate from the expert behavior. In our case, expert trajectories were generated from real life data, namely from the Interaction Dataset, as well as from a pretrained SAC (Soft Actor-Critic) agent. The expert trajectories, which are obtained in this way, represent the expert knowledge by containing many states with corresponding actions that were produced by the expert.

As mentioned previously, learning of the agent in the RL setting is driven by the rewards it receives from the environment. The amount of the reward in the Imitation Learning setting are determined based on how closely the agent mimics the expert behavior. Special in the GAIL approach is that we receive the reward from an adversarial game: The agent is represented by a generator network which is trained based on the feedback of a discriminator network. The generator produces actions for given states which are then evaluated by the discriminator. In the meantime, the discriminator is trained by feeding it with expert and agent state-action pairs for classification. This way, the generator tries to fool the discriminator, hence he aims at acting as expert-like as possible. Meanwhile, the discriminator tries to distinguish between expert and agent trajectories. Intuitively, learning converges when the generator learned to act so similarly to the expert that the discriminator cannot tell apart expert and agent trajectories any more. In game theory this point is called the Nash-equilibrium.

In practice, the implementation of a GAIL agent is usually solved in the following way for sample efficiency: The agent interacts with the environment by following its actual policy and hence agent state-action pairs are generated. These points are stored in a replay buffer for further learning. After a specified interval a training step is carried out. This training step has 2 substeps: training the discriminator and training the generator networks. 
* __Discriminator training:__ The discriminator is fed a batch of expert (from the expert trajectories) and agent (from the replay buffer) state-action pairs. It classifies all of them. Based on their true labels the loss is calculated and a gradient descent step is carried out in order to minimize the loss.
* __Generator training:__ The generator is fed a batch of states from the replay buffer and it produces actions for them. The resulting state-action pairs are fed to the discriminator for classification. The negative output of the discriminator is used as a loss for the generator network. (Close to -1 if the agent mimics the expert successfully.) The gradient of the loss is propagated all the way back to the generator network to carry out a gradient step to minimize its loss.<br>
As already stated, training runs until both, the generator and the discriminator loss, converge to a steady state value.


The training process is visualized in the following figure:


<img width=70% src="files/data/gail_overview.gif">

# Interaction Dataset
As data source, we use the Interaction Dataset: https://arxiv.org/abs/1910.03088. We are interested in the merging scenarios: 
* DR_DEU_Merging_MT
* DR_CHN_Merging_ZS

These scenarios contain a map specification and track specifications for multiple vehicles that drive on the map. The tracks represent the trajectories of the vehicles which consist of a number of consecutive recorded states. 

Have a look how the Interaction Dataset is [integrated in BARK](https://github.com/bark-simulator/bark/blob/setup_tutorials/docs/tutorials/04_interaction_dataset.ipynb). 
(Note that the dataset itself is NOT enclosed within BARK due to license limitations)

# Expert Trajectories

The Interaction Dataset of course contains trajectories of many different vehicles with different wheel bases. As there are only states recorded in the dataset, we calculate the action the vehicle has taken to go from one state to the next ourselfs.

The wheel base is used to calculate the action following the [Single Track Model](https://borrelli.me.berkeley.edu/pdfpub/IV_KinematicMPC_jason.pdf). As we are only interested in trajectories with a wheel base as of our agent, we use a fixed wheel base when calculating the actions from the successive states. This however does not restrict the accuracy of the data. We just consider that all state trajectories were carried out by the same car and calculate the actions that a car would have needed to carry out that behavior. 

As the state variables and also the actions have different magnitudes, we normalize all of them for training. The normalization of the expert trajectories takes place while loading the generated expert trajectories. The loading function also takes the current environment (BARK runtime) as an input, hence the trajectories are normalized according to the current parameters (Current state and action spaces).

You can have a look at the source code in `bark_ml.library_wrappers.lib_tf2rl.generate_expert_trajectories`

# Generate Expert Trajectories
A short example script for generating expert trajectories from the interaction dataset is shown in the following. 

In [ ]:
import os 
import bark
from pprint import pprint
from bark_ml.library_wrappers.lib_tf2rl.generate_expert_trajectories import *

tracks_folder = os.path.join(os.getcwd(), 'data')
map_file = os.path.join(os.getcwd(), 'data/DR_DEU_Merging_MT_v01_shifted.xodr')
known_key = ('DR_DEU_Merging_MT_v01_shifted', 'vehicle_tracks_013')
ego_agent = 66

param_server = create_parameter_servers_for_scenarios(map_file, tracks_folder)[known_key]
generation_params = param_server["Scenario"]["Generation"]["InteractionDatasetScenarioGeneration"]
generation_params["TrackIds"] = [63, 64, 65, 66, 67, 68]
generation_params["StartTs"] = 232000
generation_params["EndTs"] = 259000
generation_params["EgoTrackId"] = ego_agent
param_server["Scenario"]["Generation"]["InteractionDatasetScenarioGeneration"] = generation_params

In [ ]:
expert_trajectories = generate_expert_trajectories_for_scenario(param_server, sim_time_step=200, renderer="matplotlib_jupyter")

In [3]:
import pandas as pd
from IPython.display import display
from helpers import *

## The generated expert trajectories
The generated expert trajectories are stored in a dictionary with key-value pairs:
* `obs`: list, contains the observation vector for the timestep.
* `act`: list, contains the action that was carried out in that timestep.
* `next_obs`: list, contains the next observation after carrying out the action `act` in the state `obs`. 

### Format of observations

\begin{align*}
\begin{pmatrix}
x \\
y \\
\Theta \\
v
\end{pmatrix}
\text{ for the ego vehicle and the three nearest vehicles in the scene, where $x$ and $y$ are 2D coordinates, $\Theta$ is orientation and $v$ velocity.}
\end{align*}

Stored as `expert_trajectories[ego_agent]['obs']` 

In [ ]:
# Small number of observations for our agent

pd.options.display.float_format = '{:,.2f}'.format
display(observations_to_dataframe(expert_trajectories[ego_agent]['obs'][:5]))

### Format of actions
\begin{align*}
\begin{pmatrix}
a \\
\delta 
\end{pmatrix}
\text{, where $a$ is acceleration and $\delta$ is steering angle.}
\end{align*}

Stored as `expert_trajectories[ego_agent]['act']` 

In [5]:
# Small number of actions from our agent

pd.options.display.float_format = '{:,.2f}'.format
display(actions_to_dataframe(expert_trajectories[ego_agent]['act'][:5]))

,Acceleration,Steering angle
0,-0.03,0.00
1,-0.04,-0.01
2,-0.07,0.00
3,-0.10,-0.01
4,-0.12,0.00


---

# GAIL implementation
The following section describes the implementation details we chose to implement the Generative Adverserial Imitation Learning setup.

## TF2RL implementation
We use an off the shelf implementation, the library [TF2RL](https://github.com/keiohta/tf2rl). It implements several reinforcement learning algorithms in [tensorflow 2](https://www.tensorflow.org/guide/effective_tf2). 

The GAIL agent is built up as follows:
* __Generator:__ A complete DDPG agent with actor and critic networks. Both of them have 2-2 hidden layers.
* __Discriminator:__ A normal discriminator network with 2 hidden layers.

In this respect, the agent is not in the traditional GAIL setup with 2 neural networks. Instead, it has 5 networks, since the DDPG agent itself has 4 networks for greater stability during training. The DDPG agent's critic network receives the judgement of the discriminator network as the reward from the environment and its training aims to maximize this reward.

## Integration into BARK
The TF2RL based GAIL agent is integrated into the existing BARK concepts and is implemented in the following most important classes:
* __TF2RLWrapper:__ Wraps the BARK runtime to match the expectations of TF2RL about the environment. The observation and action normalization also takes place here.
    * Source: `bark_ml/library_wrappers/lib_tf2rl/tf2rl_wrapper.py`
* __BehaviorTF2RLAgent:__ Base class for TF2RL based agents.
    * Source: `bark_ml/library_wrappers/lib_tf2rl/agents/tf2rl_agent.py`
* __BehaviorGAILAgent:__ The TF2RL based GAIL agent.
    * Source: `bark_ml/library_wrappers/lib_tf2rl/agents/gail_agent.py`
* __TF2RLRunner:__ Base class for TF2RL based runners.
    * Source: `bark_ml/library_wrappers/lib_tf2rl/runners/tf2rl_runner.py`
* __GAILRunner:__ The TF2RL based GAIL runner.
    * Source: `bark_ml/library_wrappers/lib_tf2rl/runners/gail_runner.py`
    
In the followings the training process is demonstrated. Later the performance of a pre-trained agent can be visualized.

---

# Training
We will now train a GAIL agent using the implementation described above. There are several training parameters which can be set on demand:
* The number of steps to train for
* The frequency of testing during training
* The number of episodes in each testing round
* The usage of GPU accelerated calculations

In [6]:
# Customize some parameters here!

max_steps = 100000          # Number of steps to train for.
test_interval = 100         # test in every ... steps.
test_episodes = 5           # number of test episodes.
gpu = 0                     # use -1 for cpu only.

In [7]:
# imports
import os
from pathlib import Path

# BARK imports
from bark_project.bark.runtime.commons.parameters import ParameterServer
from bark.runtime.viewer.matplotlib_viewer import MPViewer
from bark.runtime.viewer.video_renderer import VideoRenderer

# BARK-ML imports
from bark_ml.environments.blueprints import ContinuousHighwayBlueprint, \
  ContinuousMergingBlueprint, ContinuousIntersectionBlueprint
from bark_ml.environments.single_agent_runtime import SingleAgentRuntime
from bark_ml.library_wrappers.lib_tf2rl.tf2rl_wrapper import TF2RLWrapper
from bark_ml.library_wrappers.lib_tf2rl.agents.gail_agent import BehaviorGAILAgent
from bark_ml.library_wrappers.lib_tf2rl.runners.gail_runner import GAILRunner
from bark_ml.library_wrappers.lib_tf2rl.load_expert_trajectories import load_expert_trajectories

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


## Training graphs

The next cell deletes the previous logs and launches tensorboard. After tensorboard has launched, please go on to the next cell and start the training. The tensorboard window refreshes itself every 30 seconds, but you can also refresh it manually in the upper right corner.

You should see the graphs for:
* **Common**
    * Common/average_step_count: The average number of steps the agent takes in the environment per scenario
    * Common/average_test_return: The average return during the test scenarios
    * Common/fps: The steps the agent takes in the environment per second
    * Common/training_return: The per scenario return of the agent during training
* **DDPG**
    * DDPG/actor_loss: The loss of the actor network
    * DDPG/critic_loss: The loss of the critic network
* **GAIL**
    * GAIL/Accuracy: The agent/expert distinguishing accuracy of the discriminator
    * GAIL/DiscriminatorLoss: The loss of the discriminator network
    * GAIL/JSdivergence: The Jensen–Shannon divergence measuring the similarity between the expert and agent

The GAIL agent should converge to a Common/average_test_return of 1, so success in every scenario it faces, after at most 10.000 scenarios.

***

Sometimes tensorboard does not refresh correctly. If you don't see all of the above graphs after 300 scenarios, please rightclick the tensorboard and click _Reload frame_.

In [8]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# launching tensorboard and deleting the previous runs logdirs:
%rm -r "data/logs"
%mkdir "data/logs"
%tensorboard --logdir "data/logs"

rm: cannot remove 'data/logs': No such file or directory


Reusing TensorBoard on port 6006 (pid 41312), started 0:26:37 ago. (Use '!kill 41312' to kill it.)

In [ ]:
# load params from the json file to create the parameter server object
params = ParameterServer(filename="data/params/gail_params.json")

# customized parameters:
params["ML"]["Settings"]["GPUUse"] = gpu
tf2rl_params = params["ML"]["GAILRunner"]["tf2rl"]
tf2rl_params["max_steps"] = max_steps
tf2rl_params["test_interval"] = test_interval
tf2rl_params["test_episodes"] = test_episodes
params["ML"]["GAILRunner"]["tf2rl"] = tf2rl_params
if params["ML"]["BehaviorGAILAgent"]["WarmUp"] > max_steps / 2:
    params["ML"]["BehaviorGAILAgent"]["WarmUp"] = max_steps / 2

# create environment
bp = ContinuousMergingBlueprint(params,
                                number_of_senarios=500,
                                random_seed=0)
env = SingleAgentRuntime(blueprint=bp,
                         render=False)

# wrapped environment for compatibility with tf2rl
wrapped_env = TF2RLWrapper(env, 
                           normalize_features=params["ML"]["Settings"]["NormalizeFeatures"])

# instantiate the GAIL agent
gail_agent = BehaviorGAILAgent(environment=wrapped_env,
                               params=params)

# load the expert trajectories
expert_trajectories, _, _ = load_expert_trajectories(
    params['ML']['ExpertTrajectories']['expert_path_dir'],
    normalize_features=params["ML"]["Settings"]["NormalizeFeatures"],
    env=env, # the unwrapped env has to be used, since that contains the unnormalized spaces.
    subset_size=params["ML"]["ExpertTrajectories"]["subset_size"]
    ) 

# instantiate a runner that is going to train the agent
runner = GAILRunner(params=params,
                 environment=wrapped_env,
                 agent=gail_agent,
                 expert_trajs=expert_trajectories)

# train the agent
runner.Train()

/home/brucknem/.cache/bazel/_bazel_brucknem/5babddc60bc977a08450e544bfdcfbae/execroot/bark_ml/bazel-out/k8-fastbuild/bin/docs/report/run.runfiles/bark_project/bark/runtime/commons/xodr_parser.py:108: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if lane.find("userData"):
12:07:30.986 [INFO] (trainer.py:65) Restored None
12:07:31.038 [INFO] (irl_trainer.py:73) Total Epi:     1 Steps:       9 Episode Steps:     9 Return: -1.0000 FPS: 182.56
12:07:31.413 [INFO] (irl_trainer.py:73) Total Epi:     2 Steps:      12 Episode Steps:     3 Return: -1.0000 FPS: 545.44
12:07:31.421 [INFO] (irl_trainer.py:73) Total Epi:     3 Steps:      16 Episode Steps:     4 Return: -1.0000 FPS: 662.53
12:07:31.442 [INFO] (irl_trainer.py:73) Total Epi:     4 Steps:      37 Episode Steps:    21 Return: -1.0000 FPS: 1116.58
12:07:31.468 [INFO] (irl_trainer.py:73) Total Epi:     5 Steps:      66 Episode Steps:    29 Return: 

12:07:32.161 [INFO] (irl_trainer.py:73) Total Epi:    66 Steps:     594 Episode Steps:     2 Return: -1.0000 FPS: 259.72
12:07:32.174 [INFO] (irl_trainer.py:73) Total Epi:    67 Steps:     607 Episode Steps:    13 Return: -1.0000 FPS: 1090.18
12:07:32.182 [INFO] (irl_trainer.py:73) Total Epi:    68 Steps:     614 Episode Steps:     7 Return: -1.0000 FPS: 1039.38
12:07:32.200 [INFO] (irl_trainer.py:73) Total Epi:    69 Steps:     633 Episode Steps:    19 Return:  1.0000 FPS: 1178.65
12:07:32.217 [INFO] (irl_trainer.py:73) Total Epi:    70 Steps:     650 Episode Steps:    17 Return: -1.0000 FPS: 1065.18
12:07:32.225 [INFO] (irl_trainer.py:73) Total Epi:    71 Steps:     653 Episode Steps:     3 Return: -1.0000 FPS: 489.06
12:07:32.240 [INFO] (irl_trainer.py:73) Total Epi:    72 Steps:     666 Episode Steps:    13 Return: -1.0000 FPS: 1002.06
12:07:32.249 [INFO] (irl_trainer.py:73) Total Epi:    73 Steps:     669 Episode Steps:     3 Return: -1.0000 FPS: 464.12
12:07:32.258 [INFO] (irl_tr


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



12:07:35.604 [INFO] (irl_trainer.py:73) Total Epi:   124 Steps:    1003 Episode Steps:     3 Return: -1.0000 FPS:  1.05
12:07:35.727 [INFO] (irl_trainer.py:73) Total Epi:   125 Steps:    1007 Episode Steps:     4 Return: -1.0000 FPS: 33.01
12:07:35.844 [INFO] (irl_trainer.py:73) Total Epi:   126 Steps:    1011 Episode Steps:     4 Return: -1.0000 FPS: 34.64
12:07:36.359 [INFO] (irl_trainer.py:73) Total Epi:   127 Steps:    1028 Episode Steps:    17 Return: -1.0000 FPS: 33.09
12:07:36.511 [INFO] (irl_trainer.py:73) Total Epi:   128 Steps:    1033 Episode Steps:     5 Return: -1.0000 FPS: 33.38
12:07:36.675 [INFO] (irl_trainer.py:73) Total Epi:   129 Steps:    1038 Episode Steps:     5 Return: -1.0000 FPS: 31.91
12:07:37.050 [INFO] (irl_trainer.py:73) Total Epi:   130 Steps:    1050 Episode Steps:    12 Return: -1.0000 FPS: 32.20
12:07:37.171 [INFO] (irl_trainer.py:73) Total Epi:   131 Steps:    1054 Episode Steps:     4 Return: -1.0000 FPS: 34.09
12:07:37.379 [INFO] (irl_trainer.py:73) 

12:07:48.811 [INFO] (irl_trainer.py:73) Total Epi:   193 Steps:    1431 Episode Steps:    16 Return: -1.0000 FPS: 36.22
12:07:49.250 [INFO] (irl_trainer.py:73) Total Epi:   194 Steps:    1446 Episode Steps:    15 Return: -1.0000 FPS: 34.31
12:07:49.726 [INFO] (irl_trainer.py:73) Total Epi:   195 Steps:    1462 Episode Steps:    16 Return:  1.0000 FPS: 33.78
12:07:50.419 [INFO] (irl_trainer.py:73) Total Epi:   196 Steps:    1484 Episode Steps:    22 Return: -1.0000 FPS: 31.81
12:07:50.720 [INFO] (irl_trainer.py:73) Total Epi:   197 Steps:    1494 Episode Steps:    10 Return: -1.0000 FPS: 33.47
12:07:51.353 [INFO] (irl_trainer.py:73) Total Epi:   198 Steps:    1514 Episode Steps:    20 Return:  1.0000 FPS: 31.67
12:07:51.940 [INFO] (irl_trainer.py:73) Total Epi:   199 Steps:    1533 Episode Steps:    19 Return: -1.0000 FPS: 32.47
12:07:52.547 [INFO] (irl_trainer.py:73) Total Epi:   200 Steps:    1553 Episode Steps:    20 Return: -1.0000 FPS: 33.03
12:07:52.798 [INFO] (irl_trainer.py:119)

...

12:44:13.387 [INFO] (irl_trainer.py:73) Total Epi:  4625 Steps:   71593 Episode Steps:    14 Return:  1.0000 FPS: 31.89
12:44:14.050 [INFO] (irl_trainer.py:73) Total Epi:  4626 Steps:   71613 Episode Steps:    20 Return:  1.0000 FPS: 30.27
12:44:14.736 [INFO] (irl_trainer.py:73) Total Epi:  4627 Steps:   71636 Episode Steps:    23 Return:  1.0000 FPS: 33.64
12:44:15.271 [INFO] (irl_trainer.py:73) Total Epi:  4628 Steps:   71653 Episode Steps:    17 Return:  1.0000 FPS: 31.83
12:44:15.624 [INFO] (irl_trainer.py:73) Total Epi:  4629 Steps:   71665 Episode Steps:    12 Return:  1.0000 FPS: 34.18
12:44:16.166 [INFO] (irl_trainer.py:73) Total Epi:  4630 Steps:   71683 Episode Steps:    18 Return:  1.0000 FPS: 33.34
12:44:16.830 [INFO] (irl_trainer.py:73) Total Epi:  4631 Steps:   71703 Episode Steps:    20 Return:  1.0000 FPS: 30.25
12:44:17.443 [INFO] (irl_trainer.py:73) Total Epi:  4632 Steps:   71724 Episode Steps:    21 Return:  1.0000 FPS: 34.37
12:44:17.817 [INFO] (irl_trainer.py:73) 

12:44:47.984 [INFO] (irl_trainer.py:73) Total Epi:  4694 Steps:   72721 Episode Steps:    19 Return:  1.0000 FPS: 32.45
12:44:48.548 [INFO] (irl_trainer.py:73) Total Epi:  4695 Steps:   72740 Episode Steps:    19 Return:  1.0000 FPS: 33.77
12:44:48.981 [INFO] (irl_trainer.py:73) Total Epi:  4696 Steps:   72755 Episode Steps:    15 Return:  1.0000 FPS: 34.79
12:44:49.454 [INFO] (irl_trainer.py:73) Total Epi:  4697 Steps:   72771 Episode Steps:    16 Return:  1.0000 FPS: 34.07
12:44:49.955 [INFO] (irl_trainer.py:73) Total Epi:  4698 Steps:   72788 Episode Steps:    17 Return:  1.0000 FPS: 34.02
12:44:50.670 [INFO] (irl_trainer.py:73) Total Epi:  4699 Steps:   72811 Episode Steps:    23 Return:  1.0000 FPS: 32.26
12:44:51.268 [INFO] (irl_trainer.py:73) Total Epi:  4700 Steps:   72831 Episode Steps:    20 Return:  1.0000 FPS: 33.53
12:44:51.499 [INFO] (irl_trainer.py:119) Evaluation Total Steps:   72831 Average Reward  1.0000 / Average Step Count  16.2 over  5 episodes
12:44:51.510 [INFO] 

12:45:23.151 [INFO] (irl_trainer.py:73) Total Epi:  4762 Steps:   73858 Episode Steps:    21 Return:  1.0000 FPS: 33.31
12:45:23.624 [INFO] (irl_trainer.py:73) Total Epi:  4763 Steps:   73874 Episode Steps:    16 Return:  1.0000 FPS: 34.01
12:45:24.325 [INFO] (irl_trainer.py:73) Total Epi:  4764 Steps:   73897 Episode Steps:    23 Return:  1.0000 FPS: 32.87
12:45:24.940 [INFO] (irl_trainer.py:73) Total Epi:  4765 Steps:   73916 Episode Steps:    19 Return:  1.0000 FPS: 30.97
12:45:25.462 [INFO] (irl_trainer.py:73) Total Epi:  4766 Steps:   73934 Episode Steps:    18 Return:  1.0000 FPS: 34.64
12:45:25.937 [INFO] (irl_trainer.py:73) Total Epi:  4767 Steps:   73950 Episode Steps:    16 Return:  1.0000 FPS: 33.84
12:45:26.450 [INFO] (irl_trainer.py:73) Total Epi:  4768 Steps:   73968 Episode Steps:    18 Return:  1.0000 FPS: 35.27
12:45:26.885 [INFO] (irl_trainer.py:73) Total Epi:  4769 Steps:   73983 Episode Steps:    15 Return:  1.0000 FPS: 34.65
12:45:27.483 [INFO] (irl_trainer.py:73) 

12:45:56.722 [INFO] (irl_trainer.py:73) Total Epi:  4830 Steps:   74949 Episode Steps:    13 Return:  1.0000 FPS: 34.55
12:45:57.287 [INFO] (irl_trainer.py:73) Total Epi:  4831 Steps:   74967 Episode Steps:    18 Return:  1.0000 FPS: 31.97
12:45:57.732 [INFO] (irl_trainer.py:73) Total Epi:  4832 Steps:   74982 Episode Steps:    15 Return:  1.0000 FPS: 33.83
12:45:58.231 [INFO] (irl_trainer.py:73) Total Epi:  4833 Steps:   74999 Episode Steps:    17 Return:  1.0000 FPS: 34.18
12:45:58.739 [INFO] (irl_trainer.py:73) Total Epi:  4834 Steps:   75014 Episode Steps:    15 Return:  1.0000 FPS: 29.69
12:45:59.369 [INFO] (irl_trainer.py:73) Total Epi:  4835 Steps:   75036 Episode Steps:    22 Return:  1.0000 FPS: 35.02
12:46:00.076 [INFO] (irl_trainer.py:73) Total Epi:  4836 Steps:   75060 Episode Steps:    24 Return:  1.0000 FPS: 34.05
12:46:00.803 [INFO] (irl_trainer.py:73) Total Epi:  4837 Steps:   75084 Episode Steps:    24 Return:  1.0000 FPS: 33.09
12:46:01.396 [INFO] (irl_trainer.py:73) 

12:46:32.994 [INFO] (irl_trainer.py:73) Total Epi:  4899 Steps:   76132 Episode Steps:    11 Return: -1.0000 FPS: 33.62
12:46:33.691 [INFO] (irl_trainer.py:73) Total Epi:  4900 Steps:   76155 Episode Steps:    23 Return:  1.0000 FPS: 33.11
12:46:33.939 [INFO] (irl_trainer.py:119) Evaluation Total Steps:   76155 Average Reward  1.0000 / Average Step Count  17.8 over  5 episodes
12:46:33.949 [INFO] (irl_trainer.py:73) Total Epi:  4901 Steps:   76156 Episode Steps:     1 Return:  1.0000 FPS:  3.94
12:46:34.291 [INFO] (irl_trainer.py:73) Total Epi:  4902 Steps:   76167 Episode Steps:    11 Return:  1.0000 FPS: 32.33
12:46:34.918 [INFO] (irl_trainer.py:73) Total Epi:  4903 Steps:   76187 Episode Steps:    20 Return:  1.0000 FPS: 32.03
12:46:35.629 [INFO] (irl_trainer.py:73) Total Epi:  4904 Steps:   76209 Episode Steps:    22 Return:  1.0000 FPS: 31.03
12:46:36.258 [INFO] (irl_trainer.py:73) Total Epi:  4905 Steps:   76230 Episode Steps:    21 Return:  1.0000 FPS: 33.45
12:46:36.796 [INFO] 

# Visualization of a trained agent 

Lastly we show you how a trained agent interacts with the environment.
Therefore, please set up the number of scenarios to visualize in the next cell.

In [ ]:
# number of scenarios to visualize
num_scenarios_to_visualize = 10

In [ ]:
# load params from the json file to create the parameter server object
params = ParameterServer(filename="data/params/gail_params.json")

# setting the path for the pretrained agent.
params["ML"]["GAILRunner"]["tf2rl"]["model_dir"] = "../../../com_github_gail_4_bark_large_data_store/pretrained_agents/gail/merging"

# customized parameters:
params["ML"]["Settings"]["GPUUse"] = gpu
tf2rl_params = params["ML"]["GAILRunner"]["tf2rl"]
tf2rl_params["max_steps"] = max_steps
tf2rl_params["test_interval"] = test_interval
tf2rl_params["test_episodes"] = test_episodes
params["ML"]["GAILRunner"]["tf2rl"] = tf2rl_params
if params["ML"]["BehaviorGAILAgent"]["WarmUp"] > max_steps / 2:
    params["ML"]["BehaviorGAILAgent"]["WarmUp"] = max_steps / 2

# create environment
bp = ContinuousMergingBlueprint(params,
                              number_of_senarios=500,
                              random_seed=0)
env = SingleAgentRuntime(blueprint=bp,
                      render=False)

# wrapped environment for compatibility with tf2rl
wrapped_env = TF2RLWrapper(env, 
normalize_features=params["ML"]["Settings"]["NormalizeFeatures"])

# instantiate the GAIL agent
gail_agent = BehaviorGAILAgent(environment=wrapped_env,
                           params=params)

# instantiate a runner that is going to train the agent.
runner = GAILRunner(params=params,
                 environment=wrapped_env,
                 agent=gail_agent,)

# Visualize the agent
runner.Visualize(num_scenarios_to_visualize, renderer="matplotlib_jupyter")